# 🚀 WhisperX API Server - Zero Cost Meeting Transcription

**Hướng dẫn sử dụng:**
1. Chọn Runtime > Change runtime type > GPU (T4)
2. Chạy tất cả cells (Runtime > Run all)
3. Copy URL ngrok hiển thị bên dưới
4. Dán vào config của Meeting App
5. Sau khi họp xong, tắt Colab

In [ ]:
# Cell 1: Cài đặt thư viện
!pip install whisperx flask pyngrok python-dotenv -q
!pip install faster-whisper -q
print("✅ Cài đặt hoàn tất!")

In [ ]:
# Cell 2: Import và Setup
from flask import Flask, request, jsonify
from pyngrok import ngrok
import whisperx
import gc
import torch
import os
from threading import Thread

# Cấu hình ngrok token (Lấy free tại: https://dashboard.ngrok.com/get-started/your-authtoken)
NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTH_TOKEN"  # ⚠️ THAY ĐỔI TOKEN NÀY

# Cấu hình HuggingFace token (Lấy tại: https://huggingface.co/settings/tokens)
HF_TOKEN = "YOUR_HF_TOKEN"  # ⚠️ THAY ĐỔI TOKEN NÀY (cần cho diarization)

app = Flask(__name__)
port = 5000

print("🔧 Đang khởi tạo...")

In [ ]:
# Cell 3: Load WhisperX Model
device = "cuda" if torch.cuda.is_available() else "cpu"
batch_size = 16
compute_type = "float16" if device == "cuda" else "int8"

print(f"🎯 Device: {device}")
print("📦 Đang load WhisperX model (large-v3)...")
model = whisperx.load_model("large-v3", device, compute_type=compute_type)
print("✅ Model loaded!")

In [ ]:
# Cell 4: API Endpoints

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({
        "status": "healthy",
        "device": device,
        "model": "whisperx-large-v3"
    })

@app.route('/transcribe', methods=['POST'])
def transcribe_audio():
    try:
        if 'file' not in request.files:
            return jsonify({"error": "No file uploaded"}), 400
        
        file = request.files['file']
        if file.filename == '':
            return jsonify({"error": "Empty filename"}), 400
        
        # Lưu file tạm
        temp_path = "/tmp/temp_audio.wav"
        file.save(temp_path)
        
        print(f"📝 Processing: {file.filename}")
        
        # 1. Load audio
        audio = whisperx.load_audio(temp_path)
        
        # 2. Transcribe
        print("🎤 Transcribing...")
        result = model.transcribe(audio, batch_size=batch_size)
        
        # 3. Align (tăng độ chính xác timestamp)
        print("⏱️ Aligning timestamps...")
        model_a, metadata = whisperx.load_align_model(
            language_code=result["language"], 
            device=device
        )
        result = whisperx.align(
            result["segments"], 
            model_a, 
            metadata, 
            audio, 
            device, 
            return_char_alignments=False
        )
        
        # 4. Diarization (phân biệt người nói)
        print("👥 Diarizing speakers...")
        if HF_TOKEN and HF_TOKEN != "YOUR_HF_TOKEN":
            diarize_model = whisperx.DiarizationPipeline(
                use_auth_token=HF_TOKEN, 
                device=device
            )
            diarize_segments = diarize_model(audio)
            result = whisperx.assign_word_speakers(diarize_segments, result)
        else:
            print("⚠️ Bỏ qua diarization (chưa có HF_TOKEN)")
        
        # Clean up
        gc.collect()
        torch.cuda.empty_cache()
        os.remove(temp_path)
        
        print("✅ Done!")
        
        return jsonify({
            "success": True,
            "language": result.get("language", "unknown"),
            "segments": result["segments"]
        })
        
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return jsonify({"error": str(e)}), 500

print("✅ API endpoints ready!")

In [ ]:
# Cell 5: Start Server với ngrok

# Setup ngrok
if NGROK_AUTH_TOKEN and NGROK_AUTH_TOKEN != "YOUR_NGROK_AUTH_TOKEN":
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    public_url = ngrok.connect(port)
    print("\n" + "="*60)
    print("🚀 SERVER ĐANG CHẠY!")
    print("="*60)
    print(f"📡 Public URL: {public_url}")
    print(f"\n✅ Copy URL này và dán vào config của Meeting App")
    print(f"\n📝 Test endpoint:")
    print(f"   GET  {public_url}/health")
    print(f"   POST {public_url}/transcribe")
    print("="*60 + "\n")
else:
    print("⚠️ CẢNH BÁO: Chưa cấu hình NGROK_AUTH_TOKEN!")
    print("👉 Lấy token tại: https://dashboard.ngrok.com/get-started/your-authtoken")
    print("👉 Sau đó sửa Cell 2 và chạy lại")

# Start Flask server
def run_flask():
    app.run(port=port, debug=False, use_reloader=False)

thread = Thread(target=run_flask)
thread.start()

print("\n💡 Server sẽ chạy cho đến khi bạn dừng Colab notebook này")
print("💡 Sau khi họp xong, nhớ Runtime > Disconnect and delete runtime")